# Setup

In [ ]:
!pip install ultralytics==8.0.196
!pip install roboflow --quiet
!pip install torch torchvision onnx onnx2keras tensorflow --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import shutil
from google.colab import files
import torch
import os
import onnx
from onnx2keras import onnx_to_keras

In [ ]:
HOME = os.getcwd()
print(HOME)

/content


# Data

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/content/datasets


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="34iN7ZopLrdgnquBf6mi")
project = rf.workspace("ariqhb").project("face-face")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Face-Face-1 in yolov8:: 100%|██████████| 10262/10262 [00:02<00:00, 4248.82it/s]


# 25-416

In [ ]:
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=1 imgsz=416 plots=True

/content
100% 21.5M/21.5M [00:00<00:00, 250MB/s]
New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/datasets/Face-Face-1/data.yaml, epochs=1, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffe

In [ ]:
!ls {HOME}/runs/detect/train/

args.yaml					    PR_curve.png	   val_batch0_pred.jpg
confusion_matrix_normalized.png			    R_curve.png		   val_batch1_labels.jpg
confusion_matrix.png				    results.csv		   val_batch1_pred.jpg
events.out.tfevents.1717166228.3eead6266ef2.1181.0  results.png		   val_batch2_labels.jpg
F1_curve.png					    train_batch0.jpg	   val_batch2_pred.jpg
labels_correlogram.jpg				    train_batch1.jpg	   weights
labels.jpg					    train_batch2.jpg
P_curve.png					    val_batch0_labels.jpg


In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
folder_path = '/content/runs/detect/train/'
zip_path = '/content/D3M4_25_416.zip'
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)
files.download(zip_path)

In [ ]:
# data validasi
%cd {HOME}
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

## Export

In [ ]:
best_model_path = '/content/runs/detect/train/weights/best.pt'
shutil.copy(best_model_path, 'D3M4_25_416.pt')
files.download('D3M4_25_416.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install onnx-keras

ERROR: Could not find a version that satisfies the requirement onnx-keras (from versions: none)
ERROR: No matching distribution found for onnx-keras


In [ ]:
import torch
import h5py

# Load your model state .pt
model_state_file = '/content/D3M4_25_416.pt'
model_state = torch.load(model_state_file, map_location=torch.device('cpu'))

# Recursively save to .h5
def save_dict_h5py(h5py_file, group_name, dic, verbose=False):
    for key, item in dic.items():
        if isinstance(item, (dict, collections.OrderedDict)):
            save_dict_h5py(h5py_file, group_name + '/' + str(key), item)
        else:
            if verbose: print(f'| writing {group_name + "/" + str(key)}')
            h5py_file[group_name + '/' + str(key)] = item

with h5py.File('/content/D3M4_25_416.pt', 'w') as f:
    save_dict_h5py(f, group_name='model_state', dic=model_state, verbose=True)

NameError: name 'collections' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/D2M4_25_416.h5')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke gambar yang akan diprediksi
image_path = '/content/datasets/Skin-Problems-Detection-9/test/images/108_jpg.rf.326dd5c8c5488c5515da1803cd8cbeb6.jpg'

# Muat gambar
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Ubah ukuran gambar sesuai dengan input yang diharapkan model
image_resized = cv2.resize(image, (sz_img, sz_img))  # Sesuaikan dengan input model Anda
image_array = np.expand_dims(image_resized, axis=0)  # Tambahkan dimensi batch

# Normalisasi gambar (jika model Anda memerlukan ini, sesuaikan dengan preprocessing yang digunakan saat pelatihan)
image_array = image_array / 255.0
# Lakukan prediksi
predictions = model.predict(image_array)

# Cetak hasil prediksi
print(predictions)
